In [ ]:
""" le 24/08/2023
Bon j'ai tout oublié mais il me semble qu'au final j'ai décidé de pas partir sur ça non ??
Je peux ajouter les zones géographiques direct aux dataset avec mon geo je sais pas quoi ?
Puis là je fais autre chose donc en fait je sais pas du tout à quoi ça me servait ces trucs là
Plus besoin de chercher le argmax si on cherche la correlation et qu'après on trouve un seuil
"""

In [2]:
""" Essai partie 3 "automatisée" par moi en récupérant des trucs d'un peu partout
Etapes principales:
1- check_FP_output_loc (ou truc appelé différemment) pour regarder dans quelle(s) zone(s) géo se trouvent les résultats FP
<!> SI intéressant quand spec001_mr > max(spec001_mr)/2 --> <!!> PAS spec001_mr, c'est aggrégation de spec001_mr sur height et time par ex
    2- trouver les zones d'intérets (spec001_mr > max(spec001_mr)/2) dans le FP output
    3- récup les données GLM, ABI etc. qui correspondent à ces zones et moments d'intérêts
    4- regarde si correspondance entre FP output et les données satellite + réseau au sol etc.
    <!> SI correspondance entre FP et données éclairs
    5- récup les informations d'intérêt (infos chimiques et électriques)
    6- (plus tard) flag les vols et tout pour la db IAGOS (mais on verra plus tard pour ça)

-?- Remarques:
- est-ce qu'on garderait pas juste les infos intéressantes (max rt) dans un nouv nc au lieu de faire argmax etc. tout le temps dans chq nouveau nb ?
    
En cours en ce moment:
- refonte de check_FP_output
""";

In [ ]:
"""
Check_FP_output goal: find which geographical area is covered by max residence time in FP output to determine which satallite data etc. should be used
NON 1- Get the extreme lon and lat for each geographical area and CONVERT IT to FP grid
NON 2- (???) Add extra column to FP output nc with geographical area indicator (like GEO_REGIONS in soft-io ?) for max res time values 
1- Open netcdf
2- Recup argmax lon-lat
3- regarder dans quelle zone géo ça se trouve
    --> how ? regarder les min et les max des lat et des lon ?
        --> si chevauche deux zones on récup les données des deux zones du coup j'imagine ?
    --> connaître les zones de couverture des satellites
    --> OBJ = savoir quelles données de satellites aller récupérer 
        --> doit savoir quel satellite couvre quelle zone
        --> trouver un moyen pas trop dégueu de déterminer quelles zones couvertes par les temps de res max FP
    
PS: pour ce qui est du spec001_mr > max/2 est-ce que j'en ai besoin ou pas?
    --> si utilise ici peut permettre de réduire la zone géographique où on veut récup des données satellites (mais est-ce que utile?)
    --> si utilise ailleurs --> comme aggregated over time and height, how can I use it to OOOOH maybe using argmax ?
        --> add time to list of dim over which finding argmax ?? or maybe just put time and not lon lat ? JSP 
""";

In [1]:
# imports
from pathlib import Path
import xarray as xr
import fpout
import xarray_extras as xrx
import common_coords

In [2]:
def check_str_to_Path(path_to_check):
    """
    Function to check if path_to_check is Path instance. 
    If not, check if path_to_check is a str and convert it to pathlib.Path object
    :param path_to_check: should be Path or str
    :return: path_to_check as a pathlib.Path object or raise exception if path_to_check not Path nor str
    """
    if not isinstance(path_to_check, Path):
        if isinstance(path_to_check, str):
            path_to_check = Path(path_to_check)
        else:
            raise TypeError(f'given path should be str or Path object, not {type(path_to_check)}')
    return path_to_check

In [3]:
""" Remarques:
- affiner le .glob(*/) si les FP output directories ont un nom spécifique après
"""
def find_FP_output_flight_dirs(flexpart_output_path):
    """
    Function returning a list of directories containing FLEXPART outputs for each flight analysed
    :param flexpart_output_path: PosixPath or str, directory containing FP outputs
    :return: sorted list of directory paths containing the FP results for each flight (one directory per flight analysed)  
    :rtype: <list>[ <PosixPath>, ... ]
    """
    # check if flexpart_output_path is instance of Path class
    flexpart_output_path = check_str_to_Path(flexpart_output_path)
    # return list of directories in flexpart_output_path
    return sorted(flexpart_output_path.glob("*/"))

In [4]:
def find_FP_output_netcdf_files(FP_out_flight_dir, nc_output_dir_name='10j_100k_output'):
    """
    Function to find path of netcdf files in a FP output result directory
    :param FP_out_flight_dir: PosixPath, path to FP output directory for the flight
    :param nc_output_dir_name: str, name of subdirectory in which netcdf file is supposed to be stored
    :return: list of netcdf files in the FP output subdirectory (usually only one nc file per flight)
    :rtype: <list>[ <PosixPath>, ... ]
    """
    FP_out_flight_dir = check_str_to_Path(FP_out_flight_dir)
    dir_path = FP_out_flight_dir / nc_output_dir_name
    # if nc_output_dir_name is in the FP output directory, we look for netcdf file
    if dir_path in sorted(FP_out_flight_dir.iterdir()):
        return sorted(dir_path.glob("*.nc"))
    else:
        raise FileNotFoundError(f'Directory {dir_path} does not exist')

#### <!> might not work if nested directories for FP output --> see 2015 directory structure in /o3p/macc flexpart output
---> dict par année par ex ??

In [5]:
def find_all_FP_output_netcdf_files(FP_out_flights_dirs):
    """
    Function to get all FP output netcdf files
    :param FP_out_flights_dirs: list of PosixPath, directory paths containing the FP results for each flight
    :return: list of all netcdf files paths
    :rtype: <list>[ <PosixPath>, ... ]
    """
    FP_out_nc_files = []
    for FP_out_dir in FP_out_flights_dirs:
        FP_out_nc_files += find_FP_output_netcdf_files(FP_out_dir)
    return sorted(FP_out_nc_files)

In [6]:
def open_FP_nc_dataset(FP_out_nc_path, chunks='auto', max_chunk_size=1e8, assign_releases_position_coords=False):
    """
    Check that FP_out_nc_path is a file ending with .nc and opens it using fpout.open_fp_dataset function
    :param FP_out_nc_path: PosixPath or str, netcdf file path
    :param max_chunk_size:
    :param chunks:
    :return: FP output dataset open with chunks
    """
    # check that we got a netcdf file
    FP_out_nc_path = check_str_to_Path(FP_out_nc_path) # convert potential str path to PosixPath
    if not FP_out_nc_path.suffix == ".nc":
        raise ValueError(f'Invalid file {FP_out_nc_path}, expecting netcdf file (path ending with .nc)')
    else:
        # use open_dataset in the future (when dev finished)
        return fpout.open_fp_dataset(FP_out_nc_path, chunks=chunks, max_chunk_size=max_chunk_size, 
                                     assign_releases_position_coords=assign_releases_position_coords) 

In [ ]:
en fait à quoi ça me sert ce truc là ? à rien non ?

In [7]:
def get_spec001_mr_argmax_dataArray(FP_ds):
    """
    
    :param FP_ds: <xarray.Dataset>, FP output dataset
    :return: <xarray.DataArray>
    """
    # get maximum residence time DataArray and get rid of the auxilary dimension "nageclass"
    FP_da = ds.spec001_mr
    FP_da = FP_da.squeeze(dim='nageclass') # <!> voir si on va pas plutôt carrément squeeze()
    # get index of max spec001_mr values over longitude and latitude dim
    spec_argmax_lon_lat = FP_da.load().argmax(dim=['longitude', 'latitude'])
    # reduce da by only keeping max values
    da_argmax = FP_da.isel(spec_argmax_lon_lat)
    # add pointspec (release n°) as coordinates
    da_argmax = da_argmax.assign_coords(pointspec=da_argmax.pointspec)
    return da_argmax

#### peut sûrement combiner les deux (is_lon_covered_by_satellite et is_lat_covered_by_satellite)

In [44]:
def is_lon_covered_by_satellite(lon_min, lon_max, satellite):
    """
    <expliquer les cas de figure ?>
    :param lon_min:
    :param lon_max:
    :param satellite: <dict> dictionary containing the satellites coverage max and min longitude and latitude values
    :return: <bool>
    """
    # lon_min inside area covered by satellite (satellite['LON_MIN'] >= lon_min <= satellite['LON_MAX'])
    lon_min_within_area = lon_min <= satellite['LON_MIN'] and lon_min <= satellite['LON_MAX'] 
    # lon_max inside area covered by satellite (satellite['LON_MIN'] >= lon_max <= satellite['LON_MAX'])
    lon_max_within_area = lon_min <= satellite['LON_MIN'] and lon_min <= satellite['LON_MAX'] 
    # satellite area inside area covered between lon_min and lon_max 
    sat_area_between_lon_min_max = lon_min < satellite['LON_MIN'] and lon_max > satellite['LON_MAX']
    
    return lon_min_within_area or lon_max_within_area or sat_area_between_lon_min_max

In [46]:
def is_lat_covered_by_satellite(lat_min, lat_max, satellite):
    """
    <expliquer les cas de figure ?>
    :param lat_min:
    :param lat_max:
    :param satellite: <dict> dictionary containing the satellites coverage max and min longitude and latitude values
    :return: <bool>
    """
    # lat_min inside area covered by satellite (satellite['LAT_MIN'] >= lat_min <= satellite['LAT_MAX'])
    lat_min_within_area = lat_min <= satellite['LAT_MIN'] and lat_min <= satellite['LAT_MAX'] 
    # lat_max inside area covered by satellite (satellite['LAT_MIN'] >= lat_max <= satellite['LAT_MAX'])
    lat_max_within_area = lat_min <= satellite['LAT_MIN'] and lat_min <= satellite['LAT_MAX'] 
    # satellite area inside area covered between lon_min and lon_max 
    sat_area_between_lat_min_max = lat_min < satellite['LAT_MIN'] and lat_max > satellite['LAT_MAX']
    
    return lat_min_within_area or lat_max_within_area or sat_area_between_lat_min_max

In [49]:
""" faire une liste des noms de satellites qui couvrent la zone sur laquelle le panache se déplace ?
--> mais là on sait toujours pas si y aura vraiment besoin de tous ces trucs
"""
def find_geographical_area_covered(lon_min, lon_max, lat_min, lat_max):
    """
    """
    for satellite_name in common_coords.SATELLITES_COVERAGE:
        satellite = common_coords.SATELLITES_COVERAGE[satellite_name]
        if is_lon_covered_by_satellite(lon_min, lon_max, satellite) and is_lat_covered_by_satellite(lat_min, lat_min, satellite):
            print(f"Area (lon_min:{lon_min.values}, lon_max:{lon_max.values}, lat_min:{lat_min.values}, lat_max:{lat_max.values}) covered by {satellite_name}:{satellite}")
            

---

In [20]:
FP_out_dirs = find_FP_output_flight_dirs('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/')
len(FP_out_dirs)

28

In [21]:
find_FP_output_netcdf_files(FP_out_dirs[0])

[PosixPath('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc')]

In [22]:
FP_out_nc_files = find_all_FP_output_netcdf_files(FP_out_dirs)
len(FP_out_nc_files)

28

In [23]:
FP_out_nc_files[0]

PosixPath('/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc')

In [24]:
ds = open_FP_nc_dataset(FP_out_nc_files[0])
ds

/home/patj/SOFT-IO-LI/src/libs/fpout/fpout/grid_time.py:176: FutureWarning: use open_dataset
  warnings.warn('use open_dataset', FutureWarning)


<xarray.Dataset>
Dimensions:     (time: 240, longitude: 720, latitude: 360, height: 27,
                 numpoint: 2, numspec: 1, nageclass: 1, pointspec: 2)
Coordinates:
  * time        (time) datetime64[ns] 2018-06-03T14:00:00 ... 2018-05-24T15:0...
  * longitude   (longitude) float32 -179.2 -178.8 -178.2 ... 179.2 179.8 180.2
  * latitude    (latitude) float32 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * height      (height) float32 500.0 1e+03 1.5e+03 ... 1.25e+04 1.3e+04 5e+04
Dimensions without coordinates: numpoint, numspec, nageclass, pointspec
Data variables: (12/15)
    RELCOM      (numpoint) |S45 dask.array<chunksize=(2,), meta=np.ndarray>
    RELLNG1     (numpoint) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    RELLNG2     (numpoint) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    RELLAT1     (numpoint) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    RELLAT2     (numpoint) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    RELZZ1      (numpoint) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    ...          ...
    RELEND      (numpoint) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
    RELPART     (numpoint) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    RELXMASS    (numspec, numpoint) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    LAGE        (nageclass) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    ORO         (latitude, longitude) int32 dask.array<chunksize=(360, 720), meta=np.ndarray>
    spec001_mr  (nageclass, pointspec, time, height, latitude, longitude) float32 dask.array<chunksize=(1, 1, 3, 27, 360, 720), meta=np.ndarray>
Attributes: (12/37)
    Conventions:            CF-1.6
    title:                  FLEXPART model output
    institution:            NILU
    source:                 Version 10.4 (2019-11-12) model output
    history:                2021-08-04 11:20 +0200  created by macc on n202
    references:             Stohl et al., Atmos. Chem. Phys., 2005, doi:10.51...
    ...                     ...
    ind_source:             2
    ind_receptor:           2
    mquasilag:              0
    nested_output:          0
    surf_only:              0
    linit_cond:             0

In [25]:
da = get_spec001_mr_argmax_dataArray(ds)
da

<xarray.DataArray 'spec001_mr' (nageclass: 1, pointspec: 2, time: 240,
                                height: 27)>
array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.5069321 , 0.5363187 , 0.55358267, ..., 0.1434387 ,
          0.11462416, 0.19211064],
         [0.5812055 , 0.49016124, 0.50742203, ..., 0.19571729,
          0.10173972, 0.21222349],
         [0.49515536, 0.5294034 , 0.6248398 , ..., 0.21748394,
          0.11640222, 0.20801452]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.6335993 , 0.869112  , 0.93641835, ..., 0.5872621 ,
          0.36202154, 0.5176597 ],
         [0.64772284, 0.7571852 , 0.9546391 , ..., 0.54447436,
          0.32363436, 0.47727302],
         [0.6483402 , 0.818442  , 1.0198704 , ..., 0.5696882 ,
          0.3341569 , 0.49877548]]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2018-06-03T14:00:00 ... 2018-05-24T15:00:00
    longitude  (nageclass, pointspec, time, height) float32 -179.2 ... 168.2
    latitude   (nageclass, pointspec, time, height) float32 -89.75 ... 15.75
  * height     (height) float32 500.0 1e+03 1.5e+03 ... 1.25e+04 1.3e+04 5e+04
  * pointspec  (pointspec) int64 0 1
Dimensions without coordinates: nageclass
Attributes:
    units:        s
    long_name:    AIRTRACER
    decay:        -0.07001485
    weightmolar:  29.0
    ohcconst:     -9e-10
    ohdconst:     -9.9
    vsetaver:     0.0

In [50]:
# faire fonction pour récup lon et lat min, max et lancer find_geographical_area_covered pour toutes les releases
for rel_nb in da.pointspec:
    lon_min = da.where(da.pointspec==rel_nb,drop=True).longitude.min()
    lon_max = da.where(da.pointspec==rel_nb,drop=True).longitude.max()
    lat_min = da.where(da.pointspec==rel_nb,drop=True).latitude.min()
    lat_max = da.where(da.pointspec==rel_nb,drop=True).latitude.max()
    find_geographical_area_covered(lon_min=lon_min, lon_max=lon_max, lat_min=lat_min, lat_max=lat_max)

Area (lon_min:-179.25, lon_max:-14.25, lat_min:-89.75, lat_max:36.25) covered by GOES_16:{'LON_MIN': -120, 'LON_MAX': -30, 'LAT_MIN': -52, 'LAT_MAX': 52}
Area (lon_min:-179.25, lon_max:180.25, lat_min:-89.75, lat_max:54.25) covered by GOES_16:{'LON_MIN': -120, 'LON_MAX': -30, 'LAT_MIN': -52, 'LAT_MAX': 52}


---

In [40]:
da.where(da.pointspec==0,drop=True).longitude.max()

<xarray.DataArray 'longitude' ()>
array(-14.25)

In [34]:
da.where(da.pointspec==1,drop=True).longitude.max()

<xarray.DataArray 'longitude' ()>
array(180.25)

In [42]:
da.where(da.pointspec==0,drop=True).longitude.min()

<xarray.DataArray 'longitude' ()>
array(-179.25)

In [43]:
da.where(da.pointspec==1,drop=True).longitude.min()

<xarray.DataArray 'longitude' ()>
array(-179.25)

In [31]:
da.where(da.pointspec==0,drop=False).longitude.max(dim='time')

<xarray.DataArray 'longitude' (nageclass: 1, pointspec: 2, height: 27)>
array([[[-20.25, -21.75, -20.75, -30.25, -35.75, -32.25, -19.75, -36.25,
         -39.75, -40.75, -31.75, -32.25, -31.75, -26.75, -18.75, -14.75,
         -21.25, -14.25, -17.25, -27.75, -44.25, -41.25, -46.75, -47.75,
         -46.75, -48.25, -62.75],
        [-37.75, -38.25, -38.25, -38.75, -37.75, -37.75, -37.25, -37.75,
         -37.25, -38.25, -40.75, -37.75,  85.75, -38.75, 174.25, 175.25,
         176.75, 176.25, 176.75, 180.25, 177.75, 100.75, 139.75, 180.25,
         177.75, 180.25, 179.75]]], dtype=float32)
Coordinates:
  * height     (height) float32 500.0 1e+03 1.5e+03 ... 1.25e+04 1.3e+04 5e+04
  * pointspec  (pointspec) int64 0 1
Dimensions without coordinates: nageclass

In [32]:
da.longitude.max(dim='height')

<xarray.DataArray 'longitude' (nageclass: 1, pointspec: 2, time: 240)>
array([[[-179.25, -179.25, -179.25, -179.25, -179.25, -179.25, -179.25,
         -179.25, -179.25, -179.25, -179.25,  -67.25,  -66.75,  -66.75,
          -66.75,  -66.25,  -65.75,  -65.25,  -65.75,  -66.25,  -66.75,
          -66.75,  -65.75,  -66.25,  -65.25,  -64.75,  -64.25,  -63.75,
          -63.75,  -64.75,  -64.25,  -63.25,  -63.25,  -62.25,  -61.75,
          -61.75,  -61.25,  -61.75,  -61.25,  -60.25,  -60.25,  -59.75,
          -59.25,  -59.75,  -59.25,  -59.25,  -58.75,  -59.25,  -57.75,
          -57.75,  -57.25,  -57.75,  -56.75,  -57.25,  -56.25,  -56.75,
          -55.75,  -55.25,  -55.25,  -55.25,  -54.75,  -54.25,  -54.25,
          -53.75,  -53.75,  -53.25,  -52.75,  -52.75,  -52.25,  -52.25,
          -52.25,  -51.25,  -50.75,  -50.75,  -50.25,  -49.75,  -49.75,
          -49.75,  -49.25,  -48.75,  -48.75,  -48.25,  -49.25,  -48.75,
          -48.25,  -48.25,  -47.75,  -47.75,  -52.25,  -51.75,  -51.75,
          -51.25,  -47.75,  -48.25,  -48.25,  -47.75,  -48.25,  -48.25,
          -42.25,  -41.75,  -41.75,  -41.25,  -40.75,  -40.25,  -40.25,
          -49.75,  -48.75,  -48.75,  -38.75,  -38.25,  -48.25,  -37.75,
          -37.75,  -37.25,  -37.25,  -36.75,  -36.25,  -35.75,  -35.25,
          -34.75,  -35.75,  -33.75,  -33.75,  -45.75,  -32.75,  -45.25,
          -32.25,  -31.75,  -31.75,  -31.75,  -43.75,  -43.75,  -43.25,
          -42.75,  -42.75,  -42.25,  -41.75,  -41.75,  -29.25,  -29.25,
...
          -82.75,  -82.75,  -82.75,  -83.25,  -83.75,  -83.75,  -84.25,
          -85.25,  -85.25,  -85.25,  -85.25,  -85.25,  -85.25,  -84.75,
          -84.75,  -84.75,  -84.75,  -84.75,  -84.75,  -84.75,  -84.75,
          -84.75,  -85.25,  -85.25,  -85.25,  -84.75,  -84.75,  -84.25,
          -84.25,  -83.75,  -83.75,  -84.25,  -84.75,  -83.25,  -83.25,
          -84.25,  -84.75,  -83.75,  -84.25,  -84.75,  -84.75,  -83.75,
          -84.25,  -83.75,  -83.75,  -83.75,  -83.75,  -83.75,  -83.75,
          -83.25,  -83.25,  -82.75,  -82.75,  -82.25,  -82.25,  -81.75,
          -81.75,  -81.75,  -81.75,  -81.25,  -81.25,  -85.75,  -81.75,
          -84.75,  -81.25,  -83.25,  -83.25,  -84.25,  180.25,  180.25,
          179.75,  179.25,  -82.75,  178.75,  -78.25,  -81.75,  -80.75,
          -80.75,  150.75,  148.75,  -78.25,  -77.75,  146.75,  145.25,
          144.25,  -78.75,  -78.25,  -77.25,  -77.25,  135.75,  134.75,
          133.75,  133.75,  132.75,  131.75,  129.75,  -76.25,  -75.25,
          -75.25,  -74.75,  -74.25,  -73.75,  -74.25,  -72.75,  180.25,
          120.75,  179.75,  106.25,  105.75,  177.75,  103.75,  100.75,
           99.25,  104.25,  103.75,  -71.25,  -71.25,  179.75,  -70.75,
          180.25,  180.25,  178.25,  177.75,  180.25,  176.25,  175.25,
          174.25,  173.75,  172.75,  174.75,  177.75,  176.75,  176.25,
          175.25,  174.25]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2018-06-03T14:00:00 ... 2018-05-24T15:00:00
  * pointspec  (pointspec) int64 0 1
Dimensions without coordinates: nageclass

In [27]:
da.where(da.pointspec==0).time.max()

<xarray.DataArray 'time' ()>
array('2018-06-03T14:00:00.000000000', dtype='datetime64[ns]')